
# Exploratory Data Analysis (EDA) – Crypto Narratives

Este notebook analiza el conjunto **`crypto_ohlc_join.csv`** con el objetivo de:

1. Validar la calidad del dataset (filas, duplicados, nulos).
2. Entender la distribución de las **narrativas** (AI, Gaming, RWA, Meme).
3. Caracterizar capitalización, volumen y precios.
4. Derivar *features* temporales (rendimientos, volatilidades).
5. Observar correlaciones y clusters preliminares.

> **Ruta esperada del CSV:** `./data/crypto_ohlc_join.csv`


In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, os

plt.style.use('classic')
sns.set_context('talk')
%matplotlib inline

DATA_PATH = './data/crypto_ohlc_join.csv'  # ajusta si tu ruta es distinta
assert os.path.exists(DATA_PATH), f'No se encontró {DATA_PATH}'
df = pd.read_csv(DATA_PATH)
print('Shape:', df.shape)
df.head()


## 1. Estructura y valores faltantes

In [ ]:
df.info(show_counts=True)
print('\nValores nulos por columna:')
print(df.isna().sum().sort_values())


## 2. Distribución por narrativa

In [ ]:
ax = df['narrative'].value_counts().plot(kind='bar', figsize=(6,4), title='Tokens por narrativa')
ax.set_ylabel('count')
plt.show()


## 3. Estadísticas de capitalización y precio (último close)

In [ ]:
latest = df.sort_values('date').groupby('id').tail(1)
stats = latest.groupby('narrative')['market_cap'].describe(percentiles=[.25,.5,.75])
display(stats.style.format('{:,.0f}'))


## 4. Trayectorias de precios (muestra aleatoria)

In [ ]:
sample_ids = np.random.choice(df['id'].unique(), size=6, replace=False)
fig, ax = plt.subplots(figsize=(10,6))
for sid in sample_ids:
    sub = df[df['id']==sid]
    ax.plot(sub['date'], sub['close'], label=sid, alpha=.7)
ax.set_title('Precios (sample)')
ax.set_ylabel('close USD')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()


## 5. Features temporales – retornos y volatilidad

In [ ]:
df['close_shift'] = df.groupby('id')['close'].shift(1)
df['ret_1d'] = df['close']/df['close_shift'] - 1
vol = (df.groupby('id')['ret_1d']
         .std(min_periods=20)
         .reset_index(name='volatility'))

vol_narr = vol.merge(df[['id','narrative']].drop_duplicates(), on='id')
sns.boxplot(data=vol_narr, x='narrative', y='volatility')
plt.title('Volatilidad diaria (~σ) por narrativa')
plt.show()


## 6. Correlación entre tokens (últimos 90 días)

In [ ]:
last90 = df[df['date'] >= df['date'].max() - pd.Timedelta('90D')]
pivot = last90.pivot(index='date', columns='id', values='close')
corr = pivot.pct_change().corr().fillna(0)
sns.heatmap(corr, cmap='coolwarm', cbar=False)
plt.title('Correlación 90 días (muestra)')
plt.show()
